In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Replicar o carregamento e limpeza inicial
FILE_PATH = 'data/adult.csv'
try:
    df = pd.read_csv(FILE_PATH, na_values='?', skipinitialspace=True)
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em {FILE_PATH}. Verifique o caminho.")
    raise

# Limpeza inicial (remoção de espaços)
df.columns = df.columns.str.strip() 
df['income'] = df['income'].str.strip() 
df = df.dropna() 

# 1. Variável Alvo (Y): Converter a renda (>50K) para 1 e (<50K) para 0
le = LabelBinarizer()
df['income'] = le.fit_transform(df['income']) 
Y = df['income'] # <-- A variável Y é definida aqui!

# 2. Variáveis Preditivas (X): Remover o alvo e colunas não necessárias/problemáticas.
# CORREÇÃO: Usando 'education.num' conforme seu DataFrame.
X = df.drop(columns=['income', 'fnlwgt', 'education.num'])

# 3. One-Hot Encoding: Transforma colunas categóricas (texto) em colunas numéricas (0 ou 1)
X = pd.get_dummies(X, drop_first=True) 

# 4. Dividir em Treino e Teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print("Dados prontos para o treinamento do modelo.")
print(f"Número de Features (Colunas): {X_train.shape[1]}") 
print(f"Tamanho do Conjunto de Teste: {X_test.shape[0]}")

Dados prontos para o treinamento do modelo.
Número de Features (Colunas): 94
Tamanho do Conjunto de Teste: 9049


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# 1. Instanciar e Treinar o Modelo (M1 - Otimizado para Precisão)
model_m1 = RandomForestClassifier(random_state=42)
# X_train e Y_train foram definidos na célula anterior
model_m1.fit(X_train, Y_train) 

# 2. Fazer Previsões no conjunto de teste
Y_pred_m1 = model_m1.predict(X_test) 

# 3. Avaliação Inicial da Precisão
print("--- Avaliação do Modelo M1 (Caixa Preta) ---")
accuracy = accuracy_score(Y_test, Y_pred_m1)
print(f"Acurácia no Conjunto de Teste: {accuracy:.4f}")

# --- Auditoria de Viés do Modelo ---

# 4. Extrair o Atributo Protegido do conjunto de teste (Gênero)
# Usamos .astype(int) para garantir que o índice seja 0 e 1, resolvendo o KeyError anterior.
sex_test = X_test['sex_Male'].astype(int) 

# 5. Criar um DataFrame temporário para o cálculo do DI
df_audit = pd.DataFrame({'sex_Male': sex_test, 'prediction': Y_pred_m1})

# 6. Contar a taxa de sucesso (prediction = 1) para cada grupo (0=Female, 1=Male)
taxa_sucesso_modelo = df_audit.groupby('sex_Male')['prediction'].mean()

# 7. Extrair as taxas (0 = Female - Não Privilegiado, 1 = Male - Privilegiado)
taxa_f_modelo = taxa_sucesso_modelo.loc[0] 
taxa_m_modelo = taxa_sucesso_modelo.loc[1] 

# 8. Calcular o DI do Modelo
di_modelo = taxa_f_modelo / taxa_m_modelo

print("\n--- Auditoria de Viés do Modelo M1 ---")
print(f"DI do Dado (Lembrete): 0.36")
print(f"DI do Modelo M1: {di_modelo:.2f}")

--- Avaliação do Modelo M1 (Caixa Preta) ---
Acurácia no Conjunto de Teste: 0.8412

--- Auditoria de Viés do Modelo M1 ---
DI do Dado (Lembrete): 0.36
DI do Modelo M1: 0.30
